In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from scipy import stats

In [2]:
data = pd.read_csv("2119-53792-0479_graph.csv")
size = data.shape[0]
MAG = data.Mag
MAGERR = data.Magerr

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MasterID  338 non-null    int64  
 1   Mag       338 non-null    float64
 2   Magerr    338 non-null    float64
 3   RA        338 non-null    float64
 4   Dec       338 non-null    float64
 5   MJD       338 non-null    float64
 6   Blend     338 non-null    int64  
dtypes: float64(5), int64(2)
memory usage: 18.6 KB


In [4]:
data.describe()

,MasterID,Mag,Magerr,RA,Dec,MJD,Blend
count,3.380000e+02,338.000000,338.000000,338.000000,338.000000,338.000000,338.0
mean,1.123069e+12,16.052692,0.069734,209.106460,25.256530,55019.655151,0.0
std,0.000000e+00,0.165166,0.006950,0.000054,0.000052,900.026772,0.0
min,1.123069e+12,15.690000,0.060000,209.106230,25.256390,53470.323460,0.0
25%,1.123069e+12,15.920000,0.070000,209.106430,25.256500,54276.222658,0.0
50%,1.123069e+12,15.990000,0.070000,209.106460,25.256520,54913.876455,0.0
75%,1.123069e+12,16.220000,0.070000,209.106490,25.256550,55939.534808,0.0
max,1.123069e+12,16.410000,0.120000,209.106630,25.256840,56464.257330,0.0


In [8]:
data.head(10)

,MasterID,Mag,Magerr,RA,Dec,MJD,Blend
0,1123069056697,15.98,0.07,209.10648,25.25651,53470.32346,0
1,1123069056697,15.95,0.07,209.10646,25.25647,53470.33163,0
2,1123069056697,15.96,0.07,209.10650,25.25648,53470.33981,0
3,1123069056697,15.97,0.07,209.10652,25.25654,53470.34792,0
4,1123069056697,15.94,0.07,209.10646,25.25655,53490.29863,0
5,1123069056697,15.92,0.07,209.10644,25.25651,53490.30787,0
6,1123069056697,15.93,0.07,209.10644,25.25651,53490.31710,0
7,1123069056697,15.91,0.07,209.10649,25.25652,53490.32638,0
8,1123069056697,15.94,0.07,209.10647,25.25654,53557.19476,0
9,1123069056697,15.94,0.07,209.10647,25.25650,53557.20077,0


In [21]:
#weighted mean
sum1 = np.sum(MAG/(MAGERR**2))
sum2 = np.sum(1/(MAGERR**2))
weighted_mean = sum1/sum2
print("weighted mean = ", round(weighted_mean,3))



#chi square 
chi_square = np.sum( ((MAG-weighted_mean)**2) / (1/(MAGERR)**2) )
print("chi square = ", round(chi_square,3))



#Standard deviation, σ w
sum3 = np.sum( (1/(MAGERR)**2) )
sum4 = np.sum( (1/(MAGERR)**2)**2 )
weighted_deviation =np.sqrt( ( sum1 / ((sum3**2)- sum4 ) * chi_square ) )
print("Standard deviation = ", round(weighted_deviation,3))



#Median absolute deviation (MAD)
#MAD = np.median(abs(MAG - np.median(MAG)))
MAD=stats.median_absolute_deviation(MAG,scale=1)
print("Median absolute deviation = ", round(MAD,3))



#Interquartile range (IQR)
IQR = stats.iqr(MAG, rng=(25, 75), nan_policy='propagate', interpolation = 'linear')
print("Interquartile range (IQR) = ", round(IQR,3))
stats.median_absolute_deviation(MAG)



#Robust median statistic (RoMS)
a = (size-1)**(-1)
RoMS = a*np.sum(abs(MAG - np.median(MAG))/ MAGERR)
print("Robust median statistic = ", round(RoMS,3))



#Normalized excess variance, σ2nxs
b = (1/(size*(weighted_mean**2)))
N_excess_variance = b*np.sum(((MAG-weighted_mean)**2) - MAGERR**2)
print("Normalized excess variance = ", N_excess_variance)



#Fvariance
Fvar=np.sqrt(N_excess_variance)
print("F variance = ", round(Fvar,3))


      
#Amplitude of variance sigma_m
c = 1/(size-1)
sigma=np.sqrt(c * np.sum((MAG-weighted_mean)**2))
epsilon = (1/size)* np.sum(((MAGERR)**2))
if sigma>epsilon:
    sigma_m = np.sqrt(sigma**2 - epsilon**2)
else:
    sigma_m=0    
print("Amplitude of variance = ", round(sigma_m,3))
    


#Peak-to-peak variability, v
v1 = max(MAG-MAGERR)
v2 = min(MAG+MAGERR)
v = (v1-v2) / (v1+v2)
print("Peak-to-peak variability = ",round(v,3))



#The von Neumann ratio η
#  η = delta_square / standard_deviation
S = 0
for i in range(1,size-1):
    A = ( MAG[i+1] - MAG[i] )**2
    S+= A/(size-1)
    
Delta_square = S
von_Neumann_ratio = Delta_square / weighted_deviation
print("The von Neumann ratio η = ",round(von_Neumann_ratio,3))


weighted mean =  16.044
chi square =  0.047
Standard deviation =  0.003
Median absolute deviation =  0.1
Interquartile range (IQR) =  0.3
Robust median statistic =  2.014
Normalized excess variance =  8.686104462752693e-05
F variance =  0.009
Amplitude of variance =  0.165
Peak-to-peak variability =  0.017
The von Neumann ratio η =  0.92
